In [1]:
# CoEPrA Example
import os
import numpy as np
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [3]:
source_dataset_path = os.path.join("book_code", "Section 4", "CoEPrA.csv")
with open(source_dataset_path) as raw_data:
    data = np.loadtxt(raw_data, delimiter=",")
print("---> Data Shape: {}".format(data.shape))

---> Data Shape: (89, 5788)


In [5]:
# Separate independent and dependent variables
X = data[:,0:5787]
y = data[:,5787]

In [6]:
# Split data into train and test
print("---> Data Split for training and testing")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("\tX_training shape '{}'".format(X_train.shape))
print("\tX_test shape '{}'".format(X_test.shape))
print("\ty_training shape '{}'".format(y_train.shape))
print("\ty_test shape '{}'".format(y_test.shape))

---> Data Split for training and testing
	X_training shape '(71, 5787)'
	X_test shape '(18, 5787)'
	y_training shape '(71,)'
	y_test shape '(18,)'
